In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = MongoClient("mongodb://localhost:27017/")

In [3]:
db = client.companies

In [4]:
collection_companies = db.clean_companies

In [5]:
query = collection_companies.find()

In [6]:
data = pd.DataFrame(query)

In [7]:
data.head()

,_id,name,category_code,number_of_employees,founded_year,deadpooled_year,description,latitude,longitude,country,state,city,monedas,total_amount_raised,geo
0,5d83a90255488441b3fff55a,Plaxo,web,50,2002.0,None,Contact Management,37.387845,-122.055197,USA,CA,Sunnyvale,Dolares estadounidenses,2.830000e+07,"{'type': 'Point', 'coordinates': [-122.055197,..."
1,5d83a90255488441b3fff55b,Kyte,games_video,40,2006.0,None,Online & Mobile Video Platform,37.788482,-122.409173,USA,CA,San Francisco,Dolares estadounidenses,2.340000e+07,"{'type': 'Point', 'coordinates': [-122.409173,..."
2,5d83a90255488441b3fff55c,Twitter,social,1300,2006.0,None,Real time communication platform,37.776805,-122.416924,USA,CA,San Francisco,Dolares estadounidenses,1.160000e+09,"{'type': 'Point', 'coordinates': [-122.4169244..."
3,5d83a90255488441b3fff55d,eBay,web,15000,1995.0,None,Online Marketplace,37.295005,-121.930035,USA,CA,San Jose,Dolares estadounidenses,6.700000e+06,"{'type': 'Point', 'coordinates': [-121.930035,..."
4,5d83a90255488441b3fff55e,Jajah,mobile,110,2005.0,None,IP Communications Platform,37.423390,-122.089951,USA,CA,Mountain View,Dolares estadounidenses,3.300000e+07,"{'type': 'Point', 'coordinates': [-122.0899512..."


In [8]:
data.shape

(200, 15)

In [9]:
# Cuento el número de empresas a un kilometro de distancia
def getCompaniesNear(geopoint, max_meters=1000):
    return len(list(db.clean_companies.find({
        "geo": {
            "$near": {
               "$geometry": geopoint,
               "$maxDistance": max_meters
             }
       }
    })))

In [10]:
# Creo una nueva columna con el número de empresas que están a un km de distancia
data["num_companies"] = data["geo"].apply(getCompaniesNear)

In [11]:
# Cuento el número de empleados a un kilometro de distancia

def getEmployees(geopoint, max_meters=1000):
    info = list(db.clean_companies.find({
        "geo": {
            "$near": {
               "$geometry": geopoint,
               "$maxDistance": max_meters
             }
       }
    }))
    
    total = 0
    for e in info:
        total += e["number_of_employees"]
    return total

In [12]:
# Creo una nueva columna con el número de empleados totales en un radio de un km de distancia

In [13]:
data["num_employees"] = data["geo"].apply(getEmployees)

In [14]:
# Cuento el total de dinero a un kilometro de distancia

def getMoney(geopoint, max_meters=1000):
    info = list(db.clean_companies.find({
        "geo": {
            "$near": {
                "$geometry": geopoint,
                "$maxDistance": max_meters
            }
        }
    }))
    
    total = 0
    for e in info:
        total += e["total_amount_raised"]
    return total

In [15]:
# Creo una nueva columna
data["total_money"] = data["geo"].apply(getMoney)

In [ ]:
################

In [18]:
zomato_key = os.getenv("zomato_key")

In [19]:
def getVeganRestaurants(lat, lon):
    
    # Hacer una llamada a la api para buscar restaurantes veganos
    # Utilizo un radio de 1 km a la redonda
    # utilizo la clave 308, la cifra que se corresponde con los locales vegetarianos
    
    headers = {
        "user-key": "{}".format(zomato_key)
    }
    url = "https://developers.zomato.com/api/v2.1/search?lat={}&lon={}&radius=1000&cuisines=308".format(lat, lon)
    response = requests.get(url, headers=headers)
    response = response.json()
    return response

In [20]:
restaurants = data.apply(lambda x: getVeganRestaurants(x["latitude"], x["longitude"]), axis=1)

In [21]:
restaurants.head()

0    {'code': 403, 'status': 'Forbidden', 'message'...
1    {'code': 403, 'status': 'Forbidden', 'message'...
2    {'code': 403, 'status': 'Forbidden', 'message'...
3    {'code': 403, 'status': 'Forbidden', 'message'...
4    {'code': 403, 'status': 'Forbidden', 'message'...
dtype: object

In [208]:
type(restaurants)

pandas.core.series.Series

In [231]:
restaurants = pd.DataFrame(restaurants)
restaurants.columns = ["restaurants"]
restaurants.head()

,restaurants
0,"{'results_found': 47, 'results_start': 0, 'res..."
1,"{'results_found': 68, 'results_start': 0, 'res..."
2,"{'results_found': 68, 'results_start': 0, 'res..."
3,"{'results_found': 46, 'results_start': 0, 'res..."
4,"{'results_found': 56, 'results_start': 0, 'res..."


In [182]:
restaurants.iloc[0]

{'results_found': 47,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': -1,
      'takeaway': -1},
     'res_id': 16861769},
    'apikey': '3d0f2fd0075d2cd6d2c5bea504648158',
    'id': '16861769',
    'name': 'Merit Vegetarian',
    'url': 'https://www.zomato.com/sunnyvale-ca/merit-vegetarian-1-sunnyvale?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '548 Lawrence Expressway, Sunnyvale 94085',
     'locality': 'Sunnyvale',
     'city': 'Sunnyvale',
     'city_id': 10823,
     'latitude': '37.3849611111',
     'longitude': '-121.9949333333',
     'zipcode': '94085',
     'country_id': 216,
     'locality_verbose': 'Sunnyvale, Sunnyvale'},
    'switch_to_order_menu': 0,
    'cuisines': 'Vegetarian',
    'timings': '11 AM to 9 PM (Mon-Sun)',
    'average_cost_for_two': 25,
    'price_range': 1,
    'currency': '$',
    'highlights': ['Dinner',
     'Takeaway Available',
     'Lunch',


In [250]:
def getInfoRestaurants(data):
    info_restaurants = []
    for rest in data:
        restaurant_dict = {
                   "name": rest["restaurants"]["restaurant"]["name"],
                   "address": rest["restaurants"]["restaurant"]["location"]["address"],
                   "lat": rest["restaurants"]["restaurant"]["location"]["latitude"],
                   "lon": rest["restaurants"]["restaurant"]["location"]["longitude"],
                   "timings": rest["restaurants"]["restaurant"]["timings"]
               }
    print(rest)
    info_restaurants.append(restaurant_dict)
    return info_restaurants


In [249]:
z = getInfoRestaurants(restaurants)

TypeError: string indices must be integers